In [4]:
import time
time.sleep(0.2)

In [1]:
import boto3
import json

bedrock_client = boto3.client('bedrock-runtime')
model_id = "anthropic.claude-3-haiku-20240307-v1:0"

user_input ="Hola"

messages = [{"role": "user","content": [{"type":"text", "text": user_input}]}]
system_prompt = "You are a helpful assistant."

body=json.dumps(
    {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 100,
        "system": system_prompt,
        "messages": messages
    }  
)  

response = bedrock_client.invoke_model(
    modelId=model_id,
    body=body
)

In [2]:
# get timestamp in milliseconds
import time
from decimal import Decimal


init_timestamp = 1719718115073
# Get current timestamp in milliseconds
timestamp_ms = int(time.time()*1000)
elapsed_time = Decimal(str((timestamp_ms - init_timestamp)/1000))

print(elapsed_time)
print ((timestamp_ms - init_timestamp)/1000)

61648.987
61648.987


In [6]:
import boto3
from boto3.dynamodb.conditions import Key

dynamodb = boto3.resource("dynamodb")

PARTIAL_MESSAGES_TABLE = "CONNECT-LLM-TbPartialMessages39A886AF-1DZ88KDGF8G20"


def save_phrase(contactId, phrase):
    print(f"New Phrase for {contactId}: {phrase}")
    table = dynamodb.Table(PARTIAL_MESSAGES_TABLE)
    timestamp_ms = int(time.time() * 1000)
    item = {"ContactId": contactId, "timestamp": timestamp_ms, "text": phrase}
    response = table.put_item(Item=item)
    return response


def get_message(contactId):
    table = dynamodb.Table(PARTIAL_MESSAGES_TABLE)
    response = table.query(
        KeyConditionExpression=Key("ContactId").eq(contactId),
        ScanIndexForward=True,
        
        Limit=1,
    )
    items = response["Items"]
    if len(items): 
        return items[0]
    return None

In [29]:
next_phrase = get_message("XYXX")
next_phrase

{'ContactId': 'XYXX',
 'text': 'Mi ultimo',
 'timestamp': Decimal('1719718559760')}

In [30]:
def delete_message(key):
    print(key)
    table = dynamodb.Table(PARTIAL_MESSAGES_TABLE)
    return table.delete_item(Key=key)

In [31]:
#next_phrase["timestamp"] = int(next_phrase["timestamp"])
say = next_phrase.get("text")
del next_phrase["text"]
delete_message(next_phrase)
{"say": say, "timestamp": int(next_phrase["timestamp"])}


{'ContactId': 'XYXX', 'timestamp': Decimal('1719718559760')}


{'say': 'Mi ultimo', 'timestamp': 1719718559760}

In [28]:
save_phrase("XYXX", "Mi ultimo")

New Phrase for XYXX: Mi ultimo


{'ResponseMetadata': {'RequestId': 'PARRMLDUL1UMEGQJA7R3EC1VN7VV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sun, 30 Jun 2024 03:35:59 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'PARRMLDUL1UMEGQJA7R3EC1VN7VV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [5]:
response_body = json.loads(response.get('body').read())

In [6]:
response_body.get('content')[0].get('text')

'¡Hola! ¿Cómo estás?'

response_body

# Tool usage

In [25]:
import logging
import boto3
import json

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

bedrock_client = boto3.client('bedrock-runtime')
model_id = "anthropic.claude-3-haiku-20240307-v1:0"
model_id            = "anthropic.claude-3-sonnet-20240229-v1:0"

def confirm_check_in(locator, last_name):
    print (locator, last_name)
    return "check-in confirmado"

In [69]:
system_prompt       = """
Hola. Tu eres Kiut, un agente IA de ayuda a de check-in para vuelos.

Aquí están las reglas importantes para la interacción:
- Inicia Saludando al pasajero y solicita su número de reserva y apellido. Es posible que el primer mensaje del usuario sea esta información.
- Tu objetivo principal es recopilar solo dos datos y nada más: el código o número de reserva (una secuencia de 6 caracteres que consta de 3
letras seguidas de 3 dígitos) y el apellido del pasajero.
- el numero de reserva puede ser dicho por el pasajero tal como se escucha, por ejemplo si dice: "a. b. c. uno dos cuatro." el numero de reserva es ABC124. También puede decir "a, v de vaca, c de casa, ciento veinti nueve" que es AVC129.
- Sea natural pero conciso en sus respuestas.
- Si la conversación comienza a desviarse del tema, cortésmente devuélvala al proceso de check-in.
- Al final de la conversación, confirma el numero de reserva y apellido del pasajero, cuando lo digas, el número de reserva debe estar encerrado en la etiqueta <prosody rate="medium"><say-as interpret-as="spell-out">{codigo_reserva}</say-as></prosody> (usa rate="medium" normalmente, si el cliente pide más lento puedes usar rate="slow")."""


tool_config = {
    "tools": [
        {
            "toolSpec": {
                "name": "confirm_check_in",
                "description": "Confirma el checkin del pasajero usando record locator y apellido",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "locator": {
                                "type": "string",
                                "description": "El record locator en formato ABC123 (tres letras y tres dígitos)",
                            },
                            "last_name": {
                                "type": "string",
                                "description": "El apellido del pasajero como Rodríguez, Vásquez o Garrido",
                            },
                        },
                        "required": ["locator", "last_name"],
                    }
                },
            }
        }
    ]
}

In [76]:
def save_phrase(contactId, phrase):
    print (f"New Phrase for {contactId}: {phrase}")

def stream_conversation(bedrock_client, model_id, system, messages, tool_config):
    #logger.info("Streaming messages with model %s", model_id)
    response = bedrock_client.converse_stream(
        system = [{"text": system}],
        modelId=model_id, messages=messages, toolConfig=tool_config
    )

    stop_reason = ""

    message = {}
    content = []
    message["content"] = content
    text = ""
    tool_use = {}

    phrases = []
    current_phrase = ""

    for chunk in response["stream"]:

        message_start       = chunk.get("messageStart")
        content_block_start = chunk.get("contentBlockStart")
        content_block_delta = chunk.get("contentBlockDelta")
        content_block_stop  =  chunk.get("contentBlockStop")
        message_stop        = chunk.get("messageStop")

        if message_start: 
            message["role"] = message_start.get("role")
        elif message_stop: 
            stop_reason = message_stop.get("stopReason")

        elif content_block_start:
            tool = content_block_start.get("start").get("toolUse")
            tool_use["toolUseId"] = tool.get("toolUseId")
            tool_use["name"] = tool.get("name")

        elif content_block_stop:
            if 'input' in tool_use:
                tool_use['input'] = json.loads(tool_use['input'])
                content.append({'toolUse': tool_use})
                #tool_use = {}
            else:
                content.append({'text': text})
                text = ''

        elif content_block_delta:
            delta = content_block_delta.get("delta")

            if delta.get("toolUse"):
                if "input" not in tool_use:
                    tool_use["input"] = ""
                tool_use["input"] += delta["toolUse"]["input"]

            elif delta.get("text"):
                delta_text = delta.get("text")
                text +=delta_text
                # print(delta_text, end="")
                current_phrase += delta_text

                last_character = delta_text[-1]
                if last_character in [".", "?", "!", "," ":", ";"]:
                    phrases.append(current_phrase)
                    save_phrase("XXXX", current_phrase.strip())
                    current_phrase = ""
    
    save_phrase("XXXX", "<fin_streaming>")
    return stop_reason, message, tool_use

def generate_streaming_text(bedrock_client, model_id, messages, tool_config):
    #logger.info("Generating text with model %s", model_id)
    new_messages = [m for m in messages]

    stop_reason, message, tool = stream_conversation( bedrock_client, model_id, system_prompt, new_messages,tool_config)
    new_messages.append(message)

    if stop_reason == 'tool_use':
        logger.info("Requesting tool %s. Request: %s",tool['name'], tool['toolUseId'])

        if tool['name'] == 'confirm_check_in':
            tool_result = {}

            res = confirm_check_in(tool['input']['locator'], tool['input']['last_name'])
            tool_result = {
                "toolUseId": tool['toolUseId'],
                "content": [{"json": {"result": res}}]
            }


            tool_result_message = {
                "role": "user",
                "content": [
                    {
                        "toolResult": tool_result

                    }
                ]
            }
            new_messages.append(tool_result_message)

            # Send the tool result to the model.
            stop_reason, message, tool = stream_conversation( bedrock_client, model_id,system_prompt, new_messages,tool_config)

            new_messages.append(message)
            print(f"output_message: {message.get("text")}")
            return new_messages

In [77]:
input_text = "Buenos días! Hola mi me llamo enrique rodriguez y mi codigo de reserva es ABC222"
messages = [{"role": "user","content": [{"text": input_text}]}]
generate_streaming_text(bedrock_client, model_id, messages, tool_config)

New Phrase for XXXX: ¡Hola Enrique!
New Phrase for XXXX: Es un gusto atenderlo.
New Phrase for XXXX: Gracias por proporcionarme su código de reserva <prosody rate="medium"><say-as interpret-as="spell-out">ABC222</say-as></prosody> y su apellido Rodríguez.


INFO:__main__:Requesting tool confirm_check_in. Request: tooluse_xJi0mwnoQhCessEJnX2DHg


New Phrase for XXXX: <fin_streaming>
ABC222 Rodriguez
New Phrase for XXXX: ¡Excelente!
New Phrase for XXXX: He confirmado su check-in para el vuelo con el código de reserva <prosody rate="medium"><say-as interpret-as="spell-out">ABC222</say-as></prosody> a nombre de Enrique Rodríguez.
New Phrase for XXXX: Esperamos que tenga un vuelo placentero.
New Phrase for XXXX: Por favor aborde por la puerta indicada en su boarding pass.
New Phrase for XXXX: Que tenga un excelente viaje.
New Phrase for XXXX: <fin_streaming>
output_message: None


[{'role': 'user',
  'content': [{'text': 'Buenos días! Hola mi me llamo enrique rodriguez y mi codigo de reserva es ABC222'}]},
 {'content': [{'text': '¡Hola Enrique! Es un gusto atenderlo. Gracias por proporcionarme su código de reserva <prosody rate="medium"><say-as interpret-as="spell-out">ABC222</say-as></prosody> y su apellido Rodríguez. \n\nPara confirmar su check-in, permítame verificar esta información con nuestro sistema:'},
   {'toolUse': {'toolUseId': 'tooluse_xJi0mwnoQhCessEJnX2DHg',
     'name': 'confirm_check_in',
     'input': {'locator': 'ABC222', 'last_name': 'Rodriguez'}}}],
  'role': 'assistant'},
 {'role': 'user',
  'content': [{'toolResult': {'toolUseId': 'tooluse_xJi0mwnoQhCessEJnX2DHg',
     'content': [{'json': {'result': 'check-in confirmado'}}]}}]},
 {'content': [{'text': '¡Excelente! He confirmado su check-in para el vuelo con el código de reserva <prosody rate="medium"><say-as interpret-as="spell-out">ABC222</say-as></prosody> a nombre de Enrique Rodríguez. 

In [58]:

input_text = "Buenos días! Hola mi me llamo enrique rodriguez y mi codigo de reserva es ABC222"
# input_text = "Hola que tal? como estamos?"
stop_reason, content, phrases, tool_use = stream_conversation(
    bedrock_client,
    model_id,
    [{"role": "user", "content": [{"text": input_text}]}],
    tool_config,
)

INFO:__main__:Streaming messages with model anthropic.claude-3-sonnet-20240229-v1:0


Muy bien Enrique, permítame confirmar su check-in con los datos que me proporcionó:

In [59]:
stop_reason, content, phrases, tool_use

('tool_use',
 {'content': [{'text': 'Muy bien Enrique, permítame confirmar su check-in con los datos que me proporcionó:'},
   {'toolUse': {'toolUseId': 'tooluse_iiH69KCgRxi_kgyFku61wA',
     'name': 'confirm_check_in',
     'input': {'last_name': 'rodriguez', 'locator': 'ABC222'}}}],
  'role': 'assistant'},
 [],
 {'toolUseId': 'tooluse_iiH69KCgRxi_kgyFku61wA',
  'name': 'confirm_check_in',
  'input': {'last_name': 'rodriguez', 'locator': 'ABC222'}})

In [37]:
input_text = "Hola que tal? como estamos?"
stream_conversation(bedrock_client, model_id, [{"role": "user","content": [{"text": input_text}]}], tool_config)

INFO:__main__:Streaming messages with model anthropic.claude-3-sonnet-20240229-v1:0


¡Hola! Estoy bien, gracias por preguntar. Soy un asistente virtual y no tengo un estado físico o emocional como tal. ¿En qué puedo ayudarte hoy?

('end_turn',
 [{'text': '¡Hola! Estoy bien, gracias por preguntar. Soy un asistente virtual y no tengo un estado físico o emocional como tal. ¿En qué puedo ayudarte hoy?'}],
 ['¡Hola!',
  ' Estoy bien, gracias por preguntar.',
  ' Soy un asistente virtual y no tengo un estado físico o emocional como tal.',
  ' ¿En qué puedo ayudarte hoy?'],
 {})

In [52]:
def generate_text(bedrock_client, model_id, tool_config, input_text):
    logger.info("Generating text with model %s", model_id)

    messages = [{"role": "user","content": [{"text": input_text}]}]

    response = bedrock_client.converse(
        modelId=model_id,
        messages=messages,
        toolConfig=tool_config
    )

    output_message = response['output']['message']
    messages.append(output_message)
    print (messages)
    stop_reason = response['stopReason']

    if stop_reason == 'tool_use':
        # Tool use requested. Call the tool and send the result to the model.
        tool_requests = response['output']['message']['content']
        for tool_request in tool_requests:
            if 'toolUse' in tool_request:
                tool = tool_request['toolUse']
                logger.info("Requesting tool %s. Request: %s",
                            tool['name'], tool['toolUseId'])

                if tool['name'] == 'confirm_check_in':
                    tool_result = {}

                    res = confirm_check_in(tool['input']['locator'], tool['input']['last_name'])
                    tool_result = {
                        "toolUseId": tool['toolUseId'],
                        "content": [{"json": {"result": res}}]
                    }
      

                    tool_result_message = {
                        "role": "user",
                        "content": [
                            {
                                "toolResult": tool_result

                            }
                        ]
                    }
                    messages.append(tool_result_message)
                    print (messages)
                    # Send the tool result to the model.
                    response = bedrock_client.converse(
                        modelId=model_id,
                        messages=messages,
                        toolConfig=tool_config
                    )
                    output_message = response['output']['message']
                    print(f"output_message: {output_message}")

    # print the final response from the model.
    for content in output_message['content']:
        print(f"{content['text']}")

In [53]:
input_text = "Hola mi me llamo enrique rodriguez y mi codigo de reserva es ABC222"


In [54]:
generate_text(bedrock_client, model_id, tool_config, input_text)


INFO:__main__:Generating text with model anthropic.claude-3-sonnet-20240229-v1:0
INFO:__main__:Requesting tool confirm_check_in. Request: tooluse_HyhsY1pWRH200oCAjSxP0w


[{'role': 'user', 'content': [{'text': 'Hola mi me llamo enrique rodriguez y mi codigo de reserva es ABC222'}]}, {'role': 'assistant', 'content': [{'text': 'Muy bien, confirmemos su check-in con esos datos:'}, {'toolUse': {'toolUseId': 'tooluse_HyhsY1pWRH200oCAjSxP0w', 'name': 'confirm_check_in', 'input': {'last_name': 'Rodriguez', 'locator': 'ABC222'}}}]}]
ABC222 Rodriguez
[{'role': 'user', 'content': [{'text': 'Hola mi me llamo enrique rodriguez y mi codigo de reserva es ABC222'}]}, {'role': 'assistant', 'content': [{'text': 'Muy bien, confirmemos su check-in con esos datos:'}, {'toolUse': {'toolUseId': 'tooluse_HyhsY1pWRH200oCAjSxP0w', 'name': 'confirm_check_in', 'input': {'last_name': 'Rodriguez', 'locator': 'ABC222'}}}]}, {'role': 'user', 'content': [{'toolResult': {'toolUseId': 'tooluse_HyhsY1pWRH200oCAjSxP0w', 'content': [{'json': {'result': 'check-in confirmado'}}]}}]}]
output_message: {'role': 'assistant', 'content': [{'text': 'El check-in ha sido confirmado exitosamente para 